# При работе с данными выяснилось, что объектов из классов:
#- товары для детей
#- украшения и аксессуары
#- электроника
#- посуда
#- бытовая техника
#- обувь
#было чрезвычайно мало и ни одна из использованных ранее моделей не могла их найти, поэтому с помощью регулярных выражений были найдены некоторые представители из вышеперечисленных классов и таким образом создался файл valid_126.csv

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
import torch
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/train.csv')
print(train.shape[0])
train.head()

1818


,text
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n..."
1,"футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/test.csv')
print(test.shape[0])
test.head()

7276


,text
0,Советую продавца
1,По вашему это платье???? Это узкая кофта !!!! За такие деньги говно!
2,"Жуткая синтетика. Неприятная ткань. Летом не поносить из-за материала. Полно торчащих ниток. Очень неудобная доставка. Лучше бы сделали так, чтобы посылка приходила на почту."
3,"Джемперок так себе на хилую четверку,запах голой синтетики,Цвет грязно синий ,на фото выглядит презентабельнее,скорее всего носить не буду"
4,"обычная х/б рубашка.не плотная,просвечивает нитки не обрезаны,в магазине вряд ли бы такую купила.качество среднее.посылка долго шла по России.пока не стирали."


In [ ]:
valid = pd.read_excel('/content/drive/MyDrive/NLP_CASE_FINAL/valid.xlsx')
print(valid.shape)
valid.head()

(100, 2)


,text,labels
0,"Долгая доставка.Плохая обработка швов,везде то...",одежда
1,Продавец мошенник говорил что товар будет отсл...,нет товара
2,Размер не соответствует. Заказали 4XL по факту...,одежда
3,"замечательно, точно как в описании, очень симп...",текстиль
4,описание точное. Качеством довольна: кофточка ...,одежда


In [ ]:
from sklearn.model_selection import train_test_split
new_train, valid = train_test_split(
    train,
    random_state=42,
    shuffle = True,
    train_size=1718,
    test_size=100
)

In [ ]:
new_train

,text
23,"Цвет черный а не темно синий , обидно. Шапка х..."
906,товар не пришел. затерялся где-то в пути прода...
485,Too long. And a bit too tight in the chest.
1268,Здравствуйте.Шапка не плохая.Тонкая и слабая р...
1629,"Платье так себе. Ткань дешевая, размер XL соот..."
...,...
1130,доставка быстрая продавец молодец.
1294,"Ждала 5 месяцев посылку ,открыла спор, продаве..."
860,"Синтетика, в центре груди проходит шов-не ровн..."
1459,Good goods and fast delivery. Thank you seller.


In [ ]:
import re

def find_by_regex(df, regex_pattern, column="text", n=100):
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    mask = df[column].apply(lambda x: bool(pattern.search(str(x))))
    result = df[mask]

    if len(result) > n:
        result = result.sample(n=n, random_state=42)

    return result

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
regex = r"" #сюда вписывал различные регулярные выражения
examples = find_by_regex(new_train, regex, n=100)
examples[["text"]]

,text
291,"Ужасная кофта. Не рекомендую. Маленькая, висит как тряпка."
1195,не пришло деньги не вернули(
1340,"размеры маек очень некорректно отображены в размерной сетке .майки короткие и широкие . подойдут только на низких и полных девушек . материал вискоза , по пошиву претензий нет ."
185,"очень тоненький. цвет не такой как на фото, намного темнее. но за такую цену 565 руб. нормально"
128,Не подошёл размер
1454,Платье соответствует описанию. Доставка быстрая. Пришло с браком - дырочка со стрелкой. Али по спору вернул деньги. С продавцом не общалась.
1012,"Все хорошо, отправили быстро. В подарок носочки ))"
1734,"Комбез как мешок если честно, очень большой+ ещё и ткань тянется"
749,"синтетика сплошная мне не очень понравился. продавец хороший, доставка быстрая"
465,"все как в описании,доставка быстрая, хороший материал."


In [ ]:
import re

def find_by_regex(df, regex_pattern, column="text", n=100):
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    mask = df[column].apply(lambda x: bool(pattern.search(str(x))))
    result = df[mask]

    if len(result) > n:
        result = result.sample(n=n, random_state=42)

    return result

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
regex = r"" #сюда вписывал различные регулярные выражения
examples = find_by_regex(test, regex, n=100)
examples[["text"]]

,text
6058,"Товар был заказан 1 марта, через пару недель трекинг показал что товар в Москве.\r\r\nв почтовом отделении не появился, продавец три раза продлял срок защиты покупателя, товар не пришел. по открытому спору вернули средства."
5441,"Очень хорошая блузка. Просто супер. Ткань натуральная. Хороший пошив, красивый цвет и размер подошёл. На ОГ 108 см ОТ 95 см. сидит свободно. С момента заказа до получения - две недели."
4475,"отличная жилетка, но надо брать на размер больше, маломерит"
3689,"Куртка пришла не того размера заказывали S а пришла M, а качество хорошее, доставили быстро."
4515,"Заказ не получили. Продавец общительный, после открытия спора деньги вернул все на следующий день. Продавца рекомендую."
185,"Шла пол месяца. Соответствует описанию, похоже шерсть присутствует потому что колется. Огорчена что с размером не угадала. Ношу хs а заказала м, рукава и длина маловата((((((((((((((((((((((хотя по размерам смотрела"
4894,"ждала пуховик,пришло вот что.\r\nоткрыла спор,жду денег назад."
3258,"Нормально сидит, по размеру идеально.Минус - платье просвечивает.\r\r\nКачество вполне соответствует цене."
4990,очень тонкая ткань. в качестве платья носить нельзя.
6142,на наш 42 xl хорошо.


С помощью регулярных выражений были найдены еще 26 примеров из редких классов и на выходе были получены файлы, которые именуются как `train_1692.csv` и `valid_126.csv` (используются в следующей большой ячейке)

# Загрузка данных, которые были преобразованы в предыдущем ноутбуке

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers huggingface_hub catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
import torch
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/train_1692.csv')
print(train.shape[0])
train.head()

1692


,text
0,товар не пришел. затерялся где-то в пути прода...
1,Too long. And a bit too tight in the chest.
2,"Платье так себе. Ткань дешевая, размер XL соот..."
3,"куртка качественная,на прохладную осень. Разме..."
4,"Я в восторге!! Качество отличное, доставка быс..."


In [5]:
valid = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/valid_126.csv')
print(valid.shape[0])
valid.head()

126


,text,labels
0,"Долгая доставка.Плохая обработка швов,везде то...",одежда
1,Продавец мошенник говорил что товар будет отсл...,нет товара
2,Размер не соответствует. Заказали 4XL по факту...,одежда
3,"замечательно, точно как в описании, очень симп...",текстиль
4,описание точное. Качеством довольна: кофточка ...,одежда


In [6]:
test = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/test.csv')
print(test.shape[0])
test.head()

7276


,text
0,Советую продавца
1,По вашему это платье???? Это узкая кофта !!!! ...
2,Жуткая синтетика. Неприятная ткань. Летом не п...
3,"Джемперок так себе на хилую четверку,запах гол..."
4,"обычная х/б рубашка.не плотная,просвечивает ни..."


# Классификация отзывов с помощью RuBERT + CatBoost

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
from sentence_transformers import SentenceTransformer
from catboost import CatBoostClassifier

model_name = "DeepPavlov/rubert-base-cased-sentence"
rubert_model = SentenceTransformer(model_name)

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
def get_embeddings(texts, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
        batch = texts[i:i+batch_size]
        batch_emb = rubert_model.encode(batch, show_progress_bar=False)
        embeddings.append(batch_emb)
    return np.vstack(embeddings)

In [10]:
embeddings_train = get_embeddings(train["text"].tolist())
embeddings_valid = get_embeddings(valid["text"].tolist())


Encoding: 100%|██████████| 4/4 [00:00<00:00,  9.79it/s]


In [11]:
clf = CatBoostClassifier(
    iterations=500,
    depth=7,
    learning_rate=0.05,
    loss_function="MultiClass",
    eval_metric="TotalF1",
    random_seed=42,
    verbose=100
)

clf.fit(embeddings_valid, valid["labels"])
pseudo_labels = clf.predict(embeddings_train).ravel()
proba = clf.predict_proba(embeddings_train)
max_proba = np.max(proba, axis=1)

threshold = 0.6
pseudo_labels[max_proba < threshold] = "unknown"

train["pseudo_label_rubert"] = pseudo_labels
train["confidence_rubert"] = max_proba

pd.set_option('display.max_colwidth', None)
sample_df = train.sample(n=50, random_state=42)[["text", "pseudo_label_rubert", "confidence_rubert"]]
sample_df.reset_index(drop=True, inplace=True)
sample_df

0:	learn: 0.7325958	total: 1.03s	remaining: 8m 35s
100:	learn: 0.9210441	total: 1m 54s	remaining: 7m 30s
200:	learn: 1.0000000	total: 2m 58s	remaining: 4m 25s
300:	learn: 1.0000000	total: 4m 1s	remaining: 2m 39s
400:	learn: 1.0000000	total: 5m 5s	remaining: 1m 15s
499:	learn: 1.0000000	total: 6m 9s	remaining: 0us


,text,pseudo_label_rubert,confidence_rubert
0,Очень прозрачное,unknown,0.417033
1,"очень разочаровало, что имеется шов, в описании его не было",одежда,0.875825
2,"очень широкая майка, как наволочка",одежда,0.836278
3,"товар пришел к другому адресату, где мой товар?",одежда,0.818180
4,товар не пришел и денег нет ((((((((((( стрееееееем продавцу фуууууууууу,unknown,0.538604
5,"спасибо , кофта нравится очень, пришло достаточно быстро))",одежда,0.855956
6,"Заказ не пришёл, и деньги тоже ни кто не вернул!!!",нет товара,0.748333
7,"товар хороший,ткань плотная, швы ровные. Размер немного большеват, но я думаю, что на вырост пойдет.)",одежда,0.967087
8,Отличная пижамка за эти деньги! На 90-67-95 подошел размер М! Пришло в спб за 8 дней.,одежда,0.929988
9,так себе,unknown,0.587052


In [12]:
print((train['pseudo_label_rubert'] == 'unknown').sum())
train['pseudo_label_rubert'].unique()

286


array(['unknown', 'нет товара', 'одежда'], dtype=object)

In [13]:
print(train.shape)
new_train = train[(train['pseudo_label_rubert']) != 'unknown']
print(new_train.shape)

(1692, 3)
(1406, 3)


In [14]:
#train[train['pseudo_label_rubert'] == 'unknown'].to_excel('candidates.xlsx')

"""
это мы получили excel-файл, где вручную разметили те объекты, в которых
класс был 'unknown'
"""

"\nэто мы получили excel-файл, где вручную разметили те объекты, в которых\nкласс был 'unknown'\n"

In [15]:
new_labels = pd.read_excel("/content/drive/MyDrive/NLP_CASE_FINAL/candidates.xlsx")
train_after_hand_classification = pd.concat([new_labels, new_train], ignore_index=True)
print(train_after_hand_classification['pseudo_label_rubert'].unique())
del train_after_hand_classification['Unnamed: 0']
train_after_hand_classification

['нет товара' 'одежда' 'текстиль' 'украшения и аксессуары'
 'товары для детей']


,text,pseudo_label_rubert,confidence_rubert
0,товар не пришел. затерялся где-то в пути продавец втечении двух дней вернул деньги. Продавец отзывчивый.,нет товара,0.553874
1,Заказала с пришла очень большая,одежда,0.538927
2,размер несоответствующий,одежда,0.583445
3,"по итогу,заказ не получила.Открыла спор,который борианули со всеми моими претензиями.Почта России похерила или с номером отслеживания какая то беда...",нет товара,0.517954
4,"Товар не пришёл, ждала два месяца. Деньги поставщике увернул только после долгой переписки и споров. Очень разочарована.",нет товара,0.502580
...,...,...,...
1687,"не совпадает с описанием,тонкая .короткая.на рукавах плохо пришитая апликация. звезды только за своевременную доставку",одежда,0.955254
1688,Все супер! М на с подойдёт,одежда,0.621826
1689,"Ждала 5 месяцев посылку ,открыла спор, продавец не хотел возвращать деньги Служба Алиэкспресс вернула .",одежда,0.875110
1690,"Синтетика, в центре груди проходит шов-не ровный, швы у гипюра кривые, швы в майке норм.",одежда,0.820450


## На данном этапе мы разметили train, теперь будем приступать к test

In [16]:
embeddings_test = get_embeddings(test["text"].tolist())
embeddings_new_train = get_embeddings(new_train['text'].tolist())

clf = CatBoostClassifier(
    iterations=800,
    depth=7,
    learning_rate=0.05,
    loss_function="MultiClass",
    eval_metric="TotalF1",
    random_seed=42,
    verbose=100
)

clf.fit(embeddings_new_train, new_train["pseudo_label_rubert"])
pseudo_labels_test = clf.predict(embeddings_test).ravel()
proba_test = clf.predict_proba(embeddings_test)
max_proba_test = np.max(proba_test, axis=1)

threshold_test = 0.75
pseudo_labels_test[max_proba_test < threshold_test] = "unknown"

test["pseudo_label_rubert"] = pseudo_labels_test
test["confidence_rubert"] = max_proba_test

pd.set_option('display.max_colwidth', None)
sample_df_test = test.sample(n=50, random_state=42)[["text", "pseudo_label_rubert", "confidence_rubert"]]
sample_df_test.reset_index(drop=True, inplace=True)
sample_df_test

Encoding: 100%|██████████| 44/44 [00:04<00:00,  9.67it/s]


0:	learn: 0.9451745	total: 826ms	remaining: 11m
100:	learn: 0.9957040	total: 1m 1s	remaining: 7m 3s
200:	learn: 1.0000000	total: 2m 3s	remaining: 6m 7s
300:	learn: 1.0000000	total: 3m 3s	remaining: 5m 4s
400:	learn: 1.0000000	total: 4m 4s	remaining: 4m 3s
500:	learn: 1.0000000	total: 5m 5s	remaining: 3m 2s
600:	learn: 1.0000000	total: 6m 5s	remaining: 2m 1s
700:	learn: 1.0000000	total: 7m 5s	remaining: 1m
799:	learn: 1.0000000	total: 8m 6s	remaining: 0us


,text,pseudo_label_rubert,confidence_rubert
0,"Товар был заказан 1 марта, через пару недель трекинг показал что товар в Москве.\r\r\nв почтовом отделении не появился, продавец три раза продлял срок защиты покупателя, товар не пришел. по открытому спору вернули средства.",одежда,0.997251
1,"Очень хорошая блузка. Просто супер. Ткань натуральная. Хороший пошив, красивый цвет и размер подошёл. На ОГ 108 см ОТ 95 см. сидит свободно. С момента заказа до получения - две недели.",одежда,0.999985
2,"отличная жилетка, но надо брать на размер больше, маломерит",одежда,0.999953
3,"Куртка пришла не того размера заказывали S а пришла M, а качество хорошее, доставили быстро.",одежда,0.999984
4,"Заказ не получили. Продавец общительный, после открытия спора деньги вернул все на следующий день. Продавца рекомендую.",нет товара,0.891847
5,"Шла пол месяца. Соответствует описанию, похоже шерсть присутствует потому что колется. Огорчена что с размером не угадала. Ношу хs а заказала м, рукава и длина маловата((((((((((((((((((((((хотя по размерам смотрела",одежда,0.999974
6,"ждала пуховик,пришло вот что.\r\nоткрыла спор,жду денег назад.",одежда,0.998928
7,"Нормально сидит, по размеру идеально.Минус - платье просвечивает.\r\r\nКачество вполне соответствует цене.",одежда,0.999985
8,очень тонкая ткань. в качестве платья носить нельзя.,одежда,0.982079
9,на наш 42 xl хорошо.,одежда,0.998293


In [17]:
print((test['pseudo_label_rubert'] == 'unknown').sum())
print(test['pseudo_label_rubert'].unique())

337
['одежда' 'нет товара' 'unknown']


In [18]:
#test[test['pseudo_label_rubert'] == 'unknown'].to_excel('/content/drive/MyDrive/NLP_case/candidates_2_labeled.xlsx')

"""
это мы получили excel-файл, где вручную разметили те объекты, в которых
класс был 'unknown'
"""

"\nэто мы получили excel-файл, где вручную разметили те объекты, в которых\nкласс был 'unknown'\n"

In [19]:
new_test = test[test['pseudo_label_rubert'] != 'unknown']

In [20]:
new_labels_test = pd.read_excel("/content/drive/MyDrive/NLP_CASE_FINAL/candidates_2_labeled.xlsx")
test_after_hand_classification = pd.concat([new_labels_test, new_test], ignore_index=True)
del test_after_hand_classification['Unnamed: 0']
test_after_hand_classification

,text,pseudo_label_rubert,confidence_rubert
0,"хорошее качество,но не хлопок",текстиль,0.550615
1,"Посылку не получили, по России не отслеживалась. Возврат денег продавец отказывался делать. 3 звезды только по тому, что посылка была отправлена.",нет товара,0.746098
2,Голимая синтетика. На фото выглядит как толстовка. Не берите,одежда,0.629128
3,сколько стоит такое и качество. но в целом нормально,текстиль,0.580259
4,ok!!!,нет товара,0.581689
...,...,...,...
7271,"Все отлично, на ОГ 86, ОТ 70, ОБ 99 рост 160 размер сел хорошо, выглядит немного оверсайз, как и хотела. Длина рукава хорошая",одежда,0.999944
7272,"так и не пришли, но деньги вернули!",нет товара,0.998988
7273,"Рубашка маломерка. XL - по меркам соответствует российскому 46-48, рукава короткие, длина - короткая. К доставке претензий нет, пришла очень быстро",одежда,0.999972
7274,на картинке совсем другая кофта и модель и ткань,одежда,0.999212


# Как итог, у нас есть размеченные с помощью **RuBERT + CatBoost** train, val, test. Теперь хочется узнать, насколько мы правильно все сделали, для этого воспользуемся **joeddav/xlm-roberta-large-xnli**

In [21]:
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1
classifier = pipeline(
    "zero-shot-classification",
    model="joeddav/xlm-roberta-large-xnli",
    device=device
)

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cuda:0


In [22]:
categories = [
    "обувь",
    "одежда",
    "текстиль",
    "товары для детей",
    "украшения и аксессуары",
    "нет товара"
]

In [23]:
def classify_text(text, threshold=0.6):
    res = classifier(text, candidate_labels=categories, multi_label=False)
    top_label = res['labels'][0]
    top_score = res['scores'][0]
    if top_score < threshold:
        return "unknown", top_score
    return top_label, top_score

In [24]:
new_valid = valid.copy()
new_valid.rename(columns={'labels': 'pseudo_label_rubert'}, inplace=True)
new_valid

,text,pseudo_label_rubert
0,"Долгая доставка.Плохая обработка швов,везде торчат нитки,неровные края.Прошивные нити темнее материала.На 42-44,рост 164 см мал",одежда
1,"Продавец мошенник говорил что товар будет отслеживаться,но в итоге дал не отслеживаемый трек номер...хотя я дополнительно заплатила...товар я,конечно же, не получила..очень жаль(",нет товара
2,Размер не соответствует. Заказали 4XL по факту оказался на 50 размер максимум.,одежда
3,"замечательно, точно как в описании, очень симпатичный комплект",текстиль
4,"описание точное. Качеством довольна: кофточка приятна телу, мягкая, на мой 44 размер села просто замечательно. Время доставки - почти месяц.",одежда
...,...,...
121,"Шапка кривая, синтетика.",украшения и аксессуары
122,очень тонкая шапка. думала будет потолще. но красивая. сидит на голове комфортно. и мне идёт. в общем нормально. но шла очень долго.,украшения и аксессуары
123,"у очков в белой оправе было деформировано стекло, все как будто ""плывет """,украшения и аксессуары
124,"на внешний вид шапка нормальная, на большой объём головы, для средних девушек будет большая. но самый главный минус, она очень холодная, на теплую осень весну.",украшения и аксессуары


In [25]:
def classify_dataframe(df, text_column="text", threshold=0):
    labels = []
    scores = []
    for text in tqdm(df[text_column].tolist(), desc="Classifying"):
        label, score = classify_text(text, threshold=threshold)
        labels.append(label)
        scores.append(score)
    df["pred_label"] = labels
    df["confidence"] = scores
    df['agree?'] = (df['pred_label'] == df['pseudo_label_rubert'])

    return df

In [26]:
train_after_hand_classification = classify_dataframe(train_after_hand_classification, text_column="text", threshold=0.6)
new_valid = classify_dataframe(new_valid, text_column="text", threshold=0.6)
test_after_hand_classification  = classify_dataframe(test_after_hand_classification, text_column="text", threshold=0.6)

Classifying: 100%|██████████| 7276/7276 [13:02<00:00,  9.29it/s]


In [27]:
print(train_after_hand_classification['agree?'].value_counts())
print(new_valid['agree?'].value_counts())
print(test_after_hand_classification['agree?'].value_counts())

#как мы видим, все плохо

agree?
False    1050
True      642
Name: count, dtype: int64
agree?
False    92
True     34
Name: count, dtype: int64
agree?
False    4628
True     2648
Name: count, dtype: int64


 # Теперь сделаем двойную проверку: будем брать в качестве меток то, что описано в ячейках

In [28]:
#train_after_hand_classification[train_after_hand_classification['pseudo_label_ensemble_final'] == 'текстиль'] #оставляем только предсказания по rubert + cb
#train_after_hand_classification[train_after_hand_classification['pred_label'] == 'текстиль'] #там, где не согласен, берем xlmroberta
#train_after_hand_classification[train_after_hand_classification['pred_label'] == 'обувь'] #обуви нет, xlmroberta предсказывает брехню, поэтому по обуви берем rubert + cb
#train_after_hand_classification[train_after_hand_classification['pred_label'] == 'товары для детей'] #лучше предсказывает rubert
#train_after_hand_classification[train_after_hand_classification['pseudo_label_ensemble_final'] == 'товары для детей'] #лучше предсказывает rubert
#train_after_hand_classification[train_after_hand_classification['pseudo_label_ensemble_final'] == 'украшения и аксессуары'] #лучше предсказывает rubert
#train_after_hand_classification[train_after_hand_classification['pred_label'] == 'украшения и аксессуары'] #берем rubert + cb
#train_after_hand_classification[train_after_hand_classification['pseudo_label_ensemble_final'] == 'одежда'] #лучше предсказывает rubert + cb
#train_after_hand_classification[train_after_hand_classification['pseudo_label_ensemble_final'] == 'нет товара'] #лучше предсказывает rubert + cb

Сверху можно поудалять '#', чтобы посмотреть, почему я сделал такие выводы. Теперь на основе того, что я вывел, разметим train

In [29]:
final_labels_train = []

for _, row in train_after_hand_classification.iterrows():
    rubert = row['pseudo_label_rubert']
    xlm = row['pred_label']
    agree = row['agree?']

    if rubert == xlm:
        final_labels_train.append(rubert)
    else:
        if 'текстиль' in [rubert, xlm]:
            if agree == False:
                final_labels_train.append(xlm)
            else:
                final_labels_train.append(rubert)
        elif 'товары для детей' in [rubert, xlm]:
            # Всегда берём RuBERT+CB
            final_labels_train.append(rubert)
        else:
            # Остальные классы — берём RuBERT+CB
            final_labels_train.append(rubert)

train_after_hand_classification['pseudo_label_ensemble_final'] = final_labels_train

In [30]:
#test_after_hand_classification[test_after_hand_classification['pseudo_label_ensemble_final'] == 'текстиль'] #оставляем rubert + cb
#test_after_hand_classification[test_after_hand_classification['pred_label'] == 'текстиль'] #берем только руберту
#test_after_hand_classification[test_after_hand_classification['pseudo_label_ensemble_final'] == 'обувь'] #нет данных
#test_after_hand_classification[test_after_hand_classification['pred_label'] == 'обувь'] #берем rubert + cb, поскольку xlmroberta реагирует на "носки" и "чулки" как на обувь
#test_after_hand_classification[test_after_hand_classification['pseudo_label_ensemble_final'] == 'товары для детей'] #можем брать и руберту, и xlmroberta
#test_after_hand_classification[test_after_hand_classification['pred_label'] == 'товары для детей'] #берем только руберту
#test_after_hand_classification[test_after_hand_classification['pseudo_label_ensemble_final'] == 'украшения и аксессуары'] #ничего нет
#test_after_hand_classification[test_after_hand_classification['pred_label'] == 'украшения и аксессуары'] #берем xlmroberta
#test_after_hand_classification[test_after_hand_classification['pseudo_label_ensemble_final'] == 'нет товара'] #берем rubert + cb
#test_after_hand_classification[test_after_hand_classification['pred_label'] == 'нет товара'] #берем rubert + cb
#для одежды берем rubert + cb

In [31]:
final_labels_test = []

for _, row in test_after_hand_classification.iterrows():
    rubert = row['pseudo_label_rubert']
    xlm = row['pred_label']
    agree = row['agree?']

    if rubert == xlm:
        final_labels_test.append(rubert)
    else:
        if 'текстиль' in [rubert, xlm]:
            final_labels_test.append(rubert)
        elif 'обувь' in [rubert, xlm]:
            final_labels_test.append(rubert)
        elif 'товары для детей' in [rubert, xlm]:
            final_labels_test.append(rubert)
        elif 'украшения и аксессуары' in [rubert, xlm]:
            final_labels_test.append(xlm)
        elif 'нет товара' in [rubert, xlm]:
            final_labels_test.append(rubert)
        else:
            final_labels_test.append(rubert)

test_after_hand_classification['pseudo_label_ensemble_final'] = final_labels_test

In [32]:
train_after_hand_classification.to_csv('FINAL_TRAIN.csv')
test_after_hand_classification.to_csv('FINAL_TEST.csv')
new_valid.to_csv('FINAL_VALID.csv')

# Подготовка финальных данных, чтобы далее сделать LoRA, а потом посчитать Weighted F1

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
!pip install -q datasets peft evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00


In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
import torch
warnings.filterwarnings('ignore')

In [36]:
final_train = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/FINAL_TRAIN.csv')
final_valid = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/FINAL_VALID.csv')
final_test = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/FINAL_TEST.csv')

In [37]:
final_train['pseudo_label_ensemble_final'] = final_train['pseudo_label_ensemble_final'].replace('unknown', 'текстиль')
final_train['pseudo_label_ensemble_final'].value_counts()

,count
pseudo_label_ensemble_final,
одежда,1197
нет товара,287
текстиль,194
украшения и аксессуары,9
товары для детей,4
обувь,1


In [38]:
final_valid = final_valid.rename(columns={'pseudo_label_rubert': 'pseudo_label_ensemble_final'})
train = pd.concat([final_train, final_valid])
del train['pseudo_label_rubert']
del train['confidence_rubert']
del train['pred_label']
del train['confidence']
del train['agree?']
del train['pseudo_label_final']
del train['Unnamed: 0']
del train['Unnamed: 0.1']
print(train['pseudo_label_ensemble_final'].value_counts())
train

pseudo_label_ensemble_final
одежда                    1273
нет товара                 310
текстиль                   202
украшения и аксессуары      19
товары для детей            12
обувь                        2
Name: count, dtype: int64


,text,pseudo_label_ensemble_final
0,товар не пришел. затерялся где-то в пути продавец втечении двух дней вернул деньги. Продавец отзывчивый.,нет товара
1,Заказала с пришла очень большая,одежда
2,размер несоответствующий,одежда
3,"по итогу,заказ не получила.Открыла спор,который борианули со всеми моими претензиями.Почта России похерила или с номером отслеживания какая то беда...",нет товара
4,"Товар не пришёл, ждала два месяца. Деньги поставщике увернул только после долгой переписки и споров. Очень разочарована.",нет товара
...,...,...
121,"Шапка кривая, синтетика.",украшения и аксессуары
122,очень тонкая шапка. думала будет потолще. но красивая. сидит на голове комфортно. и мне идёт. в общем нормально. но шла очень долго.,украшения и аксессуары
123,"у очков в белой оправе было деформировано стекло, все как будто ""плывет """,украшения и аксессуары
124,"на внешний вид шапка нормальная, на большой объём головы, для средних девушек будет большая. но самый главный минус, она очень холодная, на теплую осень весну.",украшения и аксессуары


In [39]:
!pip install deep_translator

from deep_translator import GoogleTranslator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.3 MB/s eta 0:00:00


In [40]:
import random
import time

def augment_text(text, variations=3):

    augmented_texts = []
    intermediate_languages = ['en', 'de', 'fr', 'es', 'it']

    for i in range(variations):
        try:

            intermediate_lang = random.choice(intermediate_languages)

            translated_intermediate = GoogleTranslator(
                source='ru',
                target=intermediate_lang
            ).translate(text)

            time.sleep(0.1)

            back_translated = GoogleTranslator(
                source=intermediate_lang,
                target='ru'
            ).translate(translated_intermediate)

            augmented_texts.append(back_translated)

            time.sleep(0.2)

        except Exception as e:
            print(f"Ошибка при аугментации: {e}")
            augmented_texts.append(text)

    return augmented_texts

rare_classes = ['украшения и аксессуары', 'товары для детей', 'обувь']
rare_class_samples = train[train['pseudo_label_ensemble_final'].isin(rare_classes)]

augmented_texts = []
augmented_labels = []

for _, row in tqdm(rare_class_samples.iterrows(), total=len(rare_class_samples)):
    text = row['text']
    label = row['pseudo_label_ensemble_final']

    augmented_variations = augment_text(text, variations=3)

    augmented_texts.extend(augmented_variations)
    augmented_labels.extend([label] * len(augmented_variations))

    time.sleep(0.5)

augmented_df = pd.DataFrame({
    'text': augmented_texts,
    'pseudo_label_ensemble_final': augmented_labels
})

augmented_df

100%|██████████| 33/33 [03:39<00:00,  6.65s/it]


,text,pseudo_label_ensemble_final
0,"Я заказал 3 розовых и 2 светло -розовых, в конце 5 пришли одинаковые шляпы !!!",украшения и аксессуары
1,"Я заказал 3 розы и 2 светло -розовых, в конце 5 идентичных шляп пришли !!!",украшения и аксессуары
2,"Я заказал 3 розы и 2 светло -розовых, в конце 5 идентичных шляп пришли !!!",украшения и аксессуары
3,"Good_x000d_\nМне все понравилось, но я заказал мою дочь, она оказалась большой в своих плечах. Но я подошел к себе), с SDK возникла проблема, они не могли его доставить, я ждал неделю, и в конце концов я сам взял это.",товары для детей
4,"Good_x000d_\n, Мне все понравилось, но я заказал мою дочь, она оказалась великолепной на своих плечах. Но я пришел ко мне), с SDK возникла проблема, им не удалось доставить его, я ждал неделю, и в конце концов я взял это в одиночку.",товары для детей
...,...,...
94,"Крышка нормальная для внешнего вида, для большого объема головы, для девочек в среде она будет большим. Но наименее важное, очень холодно, для теплой весны осени.",украшения и аксессуары
95,"Крышка нормальная для внешнего вида, для большого объема головы, для девочек в среде она будет большим. Но наименее важное, очень холодно, для теплой весны осени.",украшения и аксессуары
96,"Бомбардировки, для занятий спортом, я рекомендую продукты, благодаря продавцу ✌ и прибыл через две недели, хотя я выбрал оплату доставки, я теперь закажу серый",украшения и аксессуары
97,"бомбардировка, для спортивной одежды -Класс, я рекомендую товары, благодаря продавцу ✌ и пришел через две недели, хотя я выбрал оплачиваемую доставку, теперь я закажу серый",украшения и аксессуары


In [41]:
train_df_augmented = pd.concat([train, augmented_df], ignore_index=True)
train_df_augmented.to_csv('train_split_augmented.csv', index=False)

print("Augmented class counts:", train_df_augmented['pseudo_label_ensemble_final'].value_counts())

Augmented class counts: pseudo_label_ensemble_final
одежда                    1273
нет товара                 310
текстиль                   202
украшения и аксессуары      76
товары для детей            48
обувь                        8
Name: count, dtype: int64


In [42]:
train_df_augmented

,text,pseudo_label_ensemble_final
0,товар не пришел. затерялся где-то в пути продавец втечении двух дней вернул деньги. Продавец отзывчивый.,нет товара
1,Заказала с пришла очень большая,одежда
2,размер несоответствующий,одежда
3,"по итогу,заказ не получила.Открыла спор,который борианули со всеми моими претензиями.Почта России похерила или с номером отслеживания какая то беда...",нет товара
4,"Товар не пришёл, ждала два месяца. Деньги поставщике увернул только после долгой переписки и споров. Очень разочарована.",нет товара
...,...,...
1912,"Крышка нормальная для внешнего вида, для большого объема головы, для девочек в среде она будет большим. Но наименее важное, очень холодно, для теплой весны осени.",украшения и аксессуары
1913,"Крышка нормальная для внешнего вида, для большого объема головы, для девочек в среде она будет большим. Но наименее важное, очень холодно, для теплой весны осени.",украшения и аксессуары
1914,"Бомбардировки, для занятий спортом, я рекомендую продукты, благодаря продавцу ✌ и прибыл через две недели, хотя я выбрал оплату доставки, я теперь закажу серый",украшения и аксессуары
1915,"бомбардировка, для спортивной одежды -Класс, я рекомендую товары, благодаря продавцу ✌ и пришел через две недели, хотя я выбрал оплачиваемую доставку, теперь я закажу серый",украшения и аксессуары


In [43]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(
    train_df_augmented,
    test_size=0.2,
    stratify=train_df_augmented['pseudo_label_ensemble_final'],
    random_state=42
)

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(valid_df)}")
print("Train class distribution:\n", train_df['pseudo_label_ensemble_final'].value_counts(normalize=True))
print("Validation class distribution:\n", valid_df['pseudo_label_ensemble_final'].value_counts(normalize=True))

Train size: 1533
Validation size: 384
Train class distribution:
 pseudo_label_ensemble_final
одежда                    0.664057
нет товара                0.161774
текстиль                  0.105675
украшения и аксессуары    0.039791
товары для детей          0.024788
обувь                     0.003914
Name: proportion, dtype: float64
Validation class distribution:
 pseudo_label_ensemble_final
одежда                    0.664062
нет товара                0.161458
текстиль                  0.104167
украшения и аксессуары    0.039062
товары для детей          0.026042
обувь                     0.005208
Name: proportion, dtype: float64


In [44]:
train_df.to_csv('train_df_aug.csv')
valid_df.to_csv('valid_df_aug.csv')

# Делаем LoRA на аугментированных данных

In [45]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

In [46]:
dataset = load_dataset('csv', data_files={'train': '/content/train_df_aug.csv', 'validation': '/content/valid_df_aug.csv'})
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(dataset['train']['pseudo_label_ensemble_final'])
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
valid_labels = label_encoder.transform(dataset['validation']['pseudo_label_ensemble_final'])

dataset['train'] = dataset['train'].add_column('label', train_labels)
dataset['validation'] = dataset['validation'].add_column('label', valid_labels)

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['text', 'pseudo_label_ensemble_final'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')

classes = np.array(train_labels)
class_weights = compute_class_weight('balanced', classes=np.unique(classes), y=classes)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/1533 [00:00<?, ? examples/s]

Map:   0%|          | 0/384 [00:00<?, ? examples/s]

In [47]:
from collections import Counter
print(Counter(valid_labels))
print(Counter(train_labels))

Counter({np.int64(2): 255, np.int64(0): 62, np.int64(3): 40, np.int64(5): 15, np.int64(4): 10, np.int64(1): 2})
Counter({np.int64(2): 1018, np.int64(0): 248, np.int64(3): 162, np.int64(5): 61, np.int64(4): 38, np.int64(1): 6})


In [48]:
model = AutoModelForSequenceClassification.from_pretrained(
    'DeepPavlov/rubert-base-cased',
    num_labels=6,
    device_map='cuda'
)
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=['query', 'value'],
    lora_dropout=0.4,
    bias='none',
    task_type='SEQ_CLS'
)
model = get_peft_model(model, lora_config)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop('labels')
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [50]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    weighted_f1 = f1_score(labels, predictions, average='weighted')
    per_class_f1 = f1_score(labels, predictions, average=None)
    class_names = label_encoder.classes_
    for cls, f1 in zip(class_names, per_class_f1):
        print(f"Class {cls}: F1 = {f1}")
        with open('metrics.txt', 'a') as f:
            f.write(f"Epoch {trainer.state.epoch}: Class {cls}: F1 = {f1}\n")
    with open('metrics.txt', 'a') as f:
        f.write(f"Epoch {trainer.state.epoch}: Weighted F1 = {weighted_f1}\n")
    print(f"Epoch {trainer.state.epoch}: Weighted F1 = {weighted_f1}")
    return {'weighted_f1': weighted_f1}

In [55]:
#полетели
training_args = TrainingArguments(
    output_dir='./results_lora_1',
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    seed=42
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [56]:
import os
os.environ["WANDB_DISABLED"] = "true"
import wandb
wandb.init(mode="disabled")
trainer.train()

Epoch,Training Loss,Validation Loss,Weighted F1
1,1.467800,1.471222,0.356103
2,1.464300,1.273413,0.596468
3,1.347400,1.175215,0.467373
4,1.231400,1.072837,0.421932
5,0.982600,1.023092,0.605811
6,0.694700,1.027451,0.606877
7,1.054900,0.963761,0.545723
8,0.747800,0.928305,0.534736
9,0.579200,0.894945,0.618685
10,0.603400,0.912372,0.587874


Class нет товара: F1 = 0.6091954022988506
Class обувь: F1 = 0.0
Class одежда: F1 = 0.34055727554179566
Class текстиль: F1 = 0.30327868852459017
Class товары для детей: F1 = 0.0
Class украшения и аксессуары: F1 = 0.0
Epoch 1.0: Weighted F1 = 0.3561025201736204
Class нет товара: F1 = 0.6585365853658537
Class обувь: F1 = 0.0
Class одежда: F1 = 0.6725274725274726
Class текстиль: F1 = 0.2641509433962264
Class товары для детей: F1 = 0.6153846153846154
Class украшения и аксессуары: F1 = 0.0
Epoch 2.0: Weighted F1 = 0.5964678585335511
Class нет товара: F1 = 0.6753246753246753
Class обувь: F1 = 0.0
Class одежда: F1 = 0.46153846153846156
Class текстиль: F1 = 0.3238095238095238
Class товары для детей: F1 = 0.6956521739130435
Class украшения и аксессуары: F1 = 0.0
Epoch 3.0: Weighted F1 = 0.46737328191132543
Class нет товара: F1 = 0.6709677419354839
Class обувь: F1 = 0.0
Class одежда: F1 = 0.3857566765578635
Class текстиль: F1 = 0.32407407407407407
Class товары для детей: F1 = 0.5
Class украшения 

TrainOutput(global_step=1920, training_loss=0.8210218794643879, metrics={'train_runtime': 715.6527, 'train_samples_per_second': 42.842, 'train_steps_per_second': 2.683, 'total_flos': 8178817378959360.0, 'train_loss': 0.8210218794643879, 'epoch': 20.0})

In [54]:
#!rm -rf ./results_lora_1
#!rm -rf /content/best_model
#!rm -f metrics.txt misclassified.txt

In [57]:
!ls ./results_lora_1

checkpoint-1056  checkpoint-1536  checkpoint-1920  checkpoint-672
checkpoint-1152  checkpoint-1632  checkpoint-288   checkpoint-768
checkpoint-1248  checkpoint-1728  checkpoint-384   checkpoint-864
checkpoint-1344  checkpoint-1824  checkpoint-480   checkpoint-96
checkpoint-1440  checkpoint-192   checkpoint-576   checkpoint-960


# Предобученную RuBERT прогоняем через test, чтобы получить предсказания

In [58]:
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

checkpoint_path = '/content/results_lora_1/checkpoint-1632'
model = AutoModelForSequenceClassification.from_pretrained(
    'DeepPavlov/rubert-base-cased',
    num_labels=6
)
model = PeftModel.from_pretrained(model, checkpoint_path)
model = model.to('cuda')

with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

test_df = pd.read_csv('/content/drive/MyDrive/NLP_CASE_FINAL/FINAL_TEST.csv')
test_dataset = load_dataset('csv', data_files={'test': '/content/drive/MyDrive/NLP_CASE_FINAL/FINAL_TEST.csv'})

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating test split: 0 examples [00:00, ? examples/s]

In [59]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

tokenized_test = test_dataset['test'].map(tokenize_function, batched=True)
tokenized_test = tokenized_test.remove_columns(['text'])

model.eval()
predictions = []
start_time = time.time()

for i in tqdm(range(0, len(tokenized_test), 16)):
    batch = tokenized_test[i:i+16]
    inputs = {
        'input_ids': torch.tensor(batch['input_ids']).to('cuda'),
        'attention_mask': torch.tensor(batch['attention_mask']).to('cuda')
    }
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    batch_preds = torch.argmax(logits, dim=-1).cpu().numpy()
    predictions.extend(batch_preds)

elapsed_time = time.time() - start_time
avg_time_per_example = elapsed_time / len(test_df)
print(f"Average inference time per example: {avg_time_per_example:.3f} seconds")

predictions = label_encoder.inverse_transform(predictions)
predictions = ['текстиль' if pred == 'обувь' else pred for pred in predictions]

ground_truth = test_df['pseudo_label_ensemble_final'].values
weighted_f1 = f1_score(ground_truth, predictions, average='weighted')
per_class_f1 = f1_score(ground_truth, predictions, average=None)
class_names = sorted(test_df['pseudo_label_ensemble_final'].unique())
for cls, f1 in zip(class_names, per_class_f1):
    print(f"Class {cls}: F1 = {f1}")
print(f"Weighted F1-score on test set: {weighted_f1}")

test_df['predicted_label'] = predictions
test_df[['Unnamed: 0', 'text', 'pseudo_label_ensemble_final', 'predicted_label']].to_csv('test_predictions.csv', index=False)
with open('test_metrics.txt', 'w') as f:
    for cls, f1 in zip(class_names, per_class_f1):
        f.write(f"Class {cls}: F1 = {f1}\n")
    f.write(f"Weighted F1-score on test set: {weighted_f1}\n")
print("Predictions saved to test_predictions.csv")
print("Metrics saved to test_metrics.txt")

Map:   0%|          | 0/7276 [00:00<?, ? examples/s]

100%|██████████| 455/455 [03:42<00:00,  2.05it/s]

Average inference time per example: 0.031 seconds
Class нет товара: F1 = 0.6873457878040183
Class одежда: F1 = 0.7052620696784865
Class текстиль: F1 = 0.005702066999287242
Class товары для детей: F1 = 0.027777777777777776
Class украшения и аксессуары: F1 = 0.01763668430335097
Weighted F1-score on test set: 0.6983976359503813
Predictions saved to test_predictions.csv
Metrics saved to test_metrics.txt


In [60]:
a = pd.read_csv('test_predictions.csv')
a = a['predicted_label']
a.name = 'category'
a.to_csv('final_submission_all.csv')

In [61]:
!zip -r /content/my_folder.zip /content/my_folder/

from google.colab import files
files.download('/content/results_lora_1')
files.download('/content/logs')
files.download('/content/catboost_info')
files.download('/content/catboost_info')

	zip warning: name not matched: /content/my_folder/

zip error: Nothing to do! (try: zip -r /content/my_folder.zip . -i /content/my_folder/)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>